<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/2_Multiple_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge, Lasso, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go

Get API Key: http://alphavantage.co/

In [2]:
# Ambil data dari API
API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})
timeseries

{'2025-05-01': {'1. open': '16674.00000',
  '2. high': '16674.00000',
  '3. low': '16594.00000',
  '4. close': '16597.00000'},
 '2025-04-30': {'1. open': '16782.00000',
  '2. high': '16782.00000',
  '3. low': '16589.00000',
  '4. close': '16594.00000'},
 '2025-04-29': {'1. open': '16857.00000',
  '2. high': '16857.00000',
  '3. low': '16753.00000',
  '4. close': '16754.00000'},
 '2025-04-28': {'1. open': '16824.00000',
  '2. high': '16863.00000',
  '3. low': '16819.00000',
  '4. close': '16849.00000'},
 '2025-04-25': {'1. open': '16799.00000',
  '2. high': '16879.00000',
  '3. low': '16799.00000',
  '4. close': '16824.00000'},
 '2025-04-24': {'1. open': '16875.00000',
  '2. high': '16884.00000',
  '3. low': '16849.00000',
  '4. close': '16864.00000'},
 '2025-04-23': {'1. open': '16857.00000',
  '2. high': '16884.00000',
  '3. low': '16844.00000',
  '4. close': '16859.00000'},
 '2025-04-22': {'1. open': '16803.00000',
  '2. high': '16864.00000',
  '3. low': '16799.00000',
  '4. close': 

In [3]:
# Konversi ke DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)
df

,Open,High,Low,Close
2014-11-07,12120.0,12170.0,12115.0,12135.0
2014-11-10,12170.0,12170.0,12135.0,12150.0
2014-11-11,12200.0,12200.0,12200.0,12200.0
2014-11-12,12200.0,12218.0,12142.0,12197.0
2014-11-13,12197.0,12210.0,12142.0,12200.0
...,...,...,...,...
2025-04-25,16799.0,16879.0,16799.0,16824.0
2025-04-28,16824.0,16863.0,16819.0,16849.0
2025-04-29,16857.0,16857.0,16753.0,16754.0
2025-04-30,16782.0,16782.0,16589.0,16594.0


In [4]:
# Buat salinan dan fitur tambahan
df_feat = df.copy()

# Fitur hari ini hanya Open
df_feat['Open_today'] = df_feat['Open']

# Fitur hari sebelumnya (lag 1)
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)

# Hapus baris dengan missing values
df_feat.dropna(inplace=True)


In [5]:
# Pisahkan baris terakhir untuk prediksi real-time
latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

# Siapkan fitur dan target
X = df_feat[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
y = df_feat['Close']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

In [6]:
# Lihat Train Data
X_train

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2014-11-10,12170.0,12135.0,12120.0,12170.0,12115.0
2014-11-11,12200.0,12150.0,12170.0,12170.0,12135.0
2014-11-12,12200.0,12200.0,12200.0,12200.0,12200.0
2014-11-13,12197.0,12197.0,12200.0,12218.0,12142.0
2014-11-14,12200.0,12200.0,12197.0,12210.0,12142.0
...,...,...,...,...,...
2023-03-20,15360.0,15360.0,15374.0,15415.0,15339.0
2023-03-21,15360.0,15354.0,15360.0,15375.0,15339.0
2023-03-22,15345.0,15339.0,15360.0,15360.0,15329.0
2023-03-23,15344.0,15339.0,15345.0,15345.0,15339.0


In [7]:
# Lihat Test Data
X_test

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2023-03-27,15185.0,15149.0,15345.0,15345.0,15129.0
2023-03-28,15170.0,15154.0,15185.0,15185.0,15149.0
2023-03-29,15085.0,15084.0,15170.0,15189.0,15059.0
2023-03-30,15059.0,15059.0,15085.0,15109.0,15054.0
2023-03-31,15060.0,15044.0,15059.0,15090.0,15039.0
...,...,...,...,...,...
2025-04-24,16875.0,16859.0,16857.0,16884.0,16844.0
2025-04-25,16799.0,16864.0,16875.0,16884.0,16849.0
2025-04-28,16824.0,16824.0,16799.0,16879.0,16799.0
2025-04-29,16857.0,16849.0,16824.0,16863.0,16819.0


In [8]:
# Daftar model
# Daftar model
models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42)
}

# Evaluasi semua model
results = []
predictions = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    results.append({
        'Model': name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    })

# Buat DataFrame hasil evaluasi
results_df = pd.DataFrame(results)
results_df.round(4)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.448e+06, tolerance: 1.197e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.450e+06, tolerance: 1.197e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

,Model,MAE,MSE,RMSE,MAPE,R2
0,ElasticNet,39.0235,2.803490e+03,52.9480,0.2483,0.9898
1,LinearRegression,38.8757,2.802463e+03,52.9383,0.2473,0.9898
2,Ridge,38.8757,2.802463e+03,52.9383,0.2473,0.9898
3,Lasso,39.0237,2.803514e+03,52.9482,0.2483,0.9898
4,BayesianRidge,38.8630,2.802017e+03,52.9341,0.2472,0.9898
5,HuberRegressor,39.6251,2.898594e+03,53.8386,0.2523,0.9894
6,RandomForest,74.8732,1.153474e+04,107.3999,0.4696,0.9579
7,GradientBoosting,77.2489,1.234149e+04,111.0923,0.4838,0.9550
8,AdaBoost,112.4885,1.848606e+04,135.9634,0.7106,0.9326
9,ExtraTrees,69.6806,1.012341e+04,100.6152,0.4367,0.9631


In [9]:
# Inisialisasi figure
fig = go.Figure()

# Tambahkan garis aktual
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test.values,
    mode='lines',
    name='Actual',
    line=dict(width=2),
    visible=True  # selalu terlihat
))

# Tambahkan garis prediksi dari setiap model, awalnya diset tidak terlihat
for i, (name, y_pred) in enumerate(predictions.items()):
    fig.add_trace(go.Scatter(
        x=y_test.index,
        y=y_pred,
        mode='lines',
        name=f'Predicted - {name}',
        line=dict(width=2),
        visible=False  # default tidak terlihat
    ))

# Set prediksi pertama agar terlihat secara default
fig.data[1].visible = True  # trace ke-1 = prediksi pertama

# Buat tombol dropdown
dropdown_buttons = []
n_models = len(predictions)
for i, name in enumerate(predictions.keys()):
    # Visibility: True untuk trace aktual + hanya prediksi i yang True
    visibility = [True] + [j == i for j in range(n_models)]
    dropdown_buttons.append(dict(
        label=name,
        method='update',
        args=[
            {'visible': visibility},
            {'title': f'{name} Prediction of USD/IDR Closing Price'}
        ]
    ))

# Update layout dengan dropdown
fig.update_layout(
    title='Model Prediction of USD/IDR Closing Price',
    xaxis_title='Date',
    yaxis_title='Closing Price (IDR)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    width=1000,
    height=500,
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1.15,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )]
)

# Tampilkan grafik
fig.show()

In [10]:
# Pilih model terbaik secara manual
best_model_name = 'BayesianRidge'  # ← Ganti sesuai model yang Anda pilih
best_model = models[best_model_name]

In [11]:
X_latest = latest_row[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
X_latest

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2025-05-01,16674.0,16594.0,16782.0,16782.0,16589.0


In [12]:
# 🔮 Prediksi hari terakhir (yang tadi dipisahkan)

y_actual = latest_row['Close'].values[0]
y_forecast = best_model.predict(X_latest)[0]
absolute_error = abs(y_forecast - y_actual)

print("\nReal-time Forecast (Simulasi Hari Terakhir):")
print(f"Tanggal         : {latest_row.index[0].date()}")
print(f"Actual Close    : {y_actual:.2f}")
print(f"Predicted Close : {y_forecast:.2f}")
print(f"Absolute Error   : {absolute_error:.2f}")


Real-time Forecast (Simulasi Hari Terakhir):
Tanggal         : 2025-05-01
Actual Close    : 16597.00
Predicted Close : 16608.26
Absolute Error   : 11.26
